hi

# Read the Data

In [419]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier 
from sklearn.pipeline import make_pipeline
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OrdinalEncoder

In [307]:
train_df = pd.read_csv('https://raw.githubusercontent.com/sherwan-m/house_price_prediction_competition/main/housing-classification-iter6.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/sherwan-m/house_price_prediction_competition/main/test.csv')

In [140]:
test_df['Id'].head(20)

0     1461
1     1462
2     1463
3     1464
4     1465
5     1466
6     1467
7     1468
8     1469
9     1470
10    1471
11    1472
12    1473
13    1474
14    1475
15    1476
16    1477
17    1478
18    1479
19    1480
Name: Id, dtype: int64

In [141]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotArea        1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   TotalBsmtSF    1460 non-null   int64  
 3   BedroomAbvGr   1460 non-null   int64  
 4   Fireplaces     1460 non-null   int64  
 5   PoolArea       1460 non-null   int64  
 6   GarageCars     1460 non-null   int64  
 7   WoodDeckSF     1460 non-null   int64  
 8   ScreenPorch    1460 non-null   int64  
 9   Expensive      1460 non-null   int64  
 10  MSZoning       1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Heating        1460 non-null   object 
 13  Street         1460 non-null   object 
 14  CentralAir     1460 non-null   object 
 15  Foundation     1460 non-null   object 
 16  ExterQual      1460 non-null   object 
 17  ExterCond      1460 non-null   object 
 18  BsmtQual

Do we have duplicated information?

In [142]:
train_df.duplicated().sum()

0

In [308]:
X = train_df.copy()
y = X.pop('Expensive')
# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [386]:

from scipy import stats
df_filtered = train_df.copy()
for column in df_filtered.select_dtypes(include="number").columns:
    q_low = df_filtered[column].quantile(0.02)
    q_hi  = df_filtered[column].quantile(0.98)
    df_filtered1 = df_filtered[(df_filtered[column] < q_hi) & (df_filtered[column] > q_low)]
    
    # df_filtered[np.abs(df_filtered[column]-df_filtered[column].mean()) <= (3*df_filtered[column].std())]
    
    # df_filtered[(np.abs(stats.zscore(df_filtered[column])) < 3)]
    
df_y1 = df_filtered1.pop('Expensive')

In [356]:
from scipy import stats
df_filtered = train_df.copy()
for column in df_filtered.select_dtypes(include="number").columns:
    # q_low = df_filtered[column].quantile(0.01)
    # q_hi  = df_filtered[column].quantile(0.99)
    # df_filtered = X[(X[column] < q_hi) & (X[column] > q_low)]
    
    df_filtered[np.abs(df_filtered[column]-df_filtered[column].mean()) <= (3*df_filtered[column].std())]
    
df_y = df_filtered.pop('Expensive')

In [382]:
df_filtered1.shape

(1314, 80)

In [314]:
train_df.describe()

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,...,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,70.049958,1057.429452,2.866438,0.613014,2.758904,1.767123,94.244521,15.060959,0.148630,...,6.517808,1978.506164,472.980137,46.660274,21.954110,3.409589,43.489041,6.321918,2007.815753,730.500000
std,9981.264932,24.284752,438.705324,0.815778,0.644666,40.177307,0.747315,125.338794,55.757415,0.355845,...,1.625393,24.689725,213.804841,66.256028,61.119149,29.317331,496.123024,2.703626,1.328095,421.610009
min,1300.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,1.000000
25%,7553.500000,59.000000,795.750000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,5.000000,1961.000000,334.500000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,365.750000
50%,9478.500000,69.000000,991.500000,3.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,6.000000,1980.000000,480.000000,25.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,730.500000
75%,11601.500000,80.000000,1298.250000,3.000000,1.000000,0.000000,2.000000,168.000000,0.000000,0.000000,...,7.000000,2002.000000,576.000000,68.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,1095.250000
max,215245.000000,313.000000,6110.000000,8.000000,3.000000,738.000000,4.000000,857.000000,480.000000,1.000000,...,14.000000,2010.000000,1418.000000,547.000000,552.000000,508.000000,15500.000000,12.000000,2010.000000,1460.000000


## Feature Selection: 
1. convert non_numerics columns to numeric
2. choose the helpful featuers

Scikit-Learn models cannot deal with categorical features

choose the helpful featuers

In [144]:
len(train_df.select_dtypes(include="number").columns)

38

In [328]:
num_columns_v1 = ['LotFrontage', 'LotArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','1stFlrSF', '2ndFlrSF',
                  'LowQualFinSF', 'GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
                  'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch','PoolArea','MiscVal',
                  'BsmtUnfSF','MoSold', 'YrSold' ]
cat_onehot_calumns_v1 = ['MSZoning', 'Street', 'LandContour','Neighborhood','Condition1','Condition2','BldgType' ,'HouseStyle','RoofStyle','RoofMatl',
                         'Exterior1st','MasVnrType','Foundation','Heating','CentralAir','Electrical', 'Functional', 'GarageType','GarageFinish', 'Fence','MiscFeature',
                         'SaleType', 'SaleCondition','Exterior2nd', "Alley"]
cat_ordinal_calumns_v1 = ['MSSubClass', 'LotShape', 'Utilities','LotConfig','LandSlope','ExterQual', 'ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                          'BsmtFinType2','HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual','GarageCond', 'PavedDrive', 'PoolQC']
MSSubClass_scores = [20,30,40,45,50,60,70,75,80,85,90,120,150,160,180,190]
LotShape_scores = [ 'NA','IR3','IR2', 'IR1','Reg']
Utilities_scores = ['NA','ELO', 'NoSeWa', 'NoSewr','AllPub']
LotConfig_scores = ['NA', 'Inside', 'Corner', 'CulDSac', 'FR2','FR3']
LandSlope_scores = ['NA', 'Sev', 'Mod', 'Gtl' ]
ExterQual_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
ExterCond_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
BsmtQual_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
BsmtCond_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
BsmtExposure_scores = ['NA','No','Mn','Av','Gd']
BsmtFinType1_scores = ['NA', 'Unf', 'LwQ','Rec', 'BLQ', 'ALQ','GLQ']
BsmtFinType2_scores = ['NA', 'Unf', 'LwQ','Rec', 'BLQ', 'ALQ','GLQ']
HeatingQC_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
KitchenQual_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
FireplaceQu_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
GarageQual_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
GarageCond_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
PavedDrive_scores =['NA','N','P','Y']
PoolQC_scores = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']

In [170]:
num_columns_v2 = ['LotFrontage', 'LotArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','1stFlrSF','2ndFlrSF',
                  'LowQualFinSF', 'GrLivArea','BsmtFullBath','BsmtHalfBath', 'FullBath', 'HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces', 'GarageYrBlt',
                  'GarageCars', 'GarageArea','WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch' ,'ScreenPorch', 'PoolArea', 'MiscVal', 
                  'BsmtUnfSF','MoSold', 'YrSold' ]
cat_onehot_calumns_v2 = ['MSZoning','MSSubClass', 'Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle',
                         'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','CentralAir','Electrical','Functional', 'GarageType','GarageFinish', 'PavedDrive',
                         'Fence','MiscFeature', 'SaleType','SaleCondition', "Alley"]
cat_ordinal_calumns_v2 = ['ExterQual', 'ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2', 'HeatingQC','KitchenQual','FireplaceQu', 'GarageQual',
                          'GarageCond','PoolQC']


In [149]:
drop_columns= ["Alley",'BsmtUnfSF','MoSold', 'YrSold' ]
weight_columns = ['OverallQual','OverallCond','YearBuilt','YearRemodAdd']

In [455]:
# Version 1

min_frequency= int(len(X)*0.002)
max_categories = int(len(X)*0.995)

numeric_pipe_v1 = make_pipeline(
    IterativeImputer(random_state=0, initial_strategy="mean"))
# create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
cat_onehot_pipe_v1 = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(min_frequency = min_frequency, max_categories=max_categories, handle_unknown='infrequent_if_exist', drop="first"),
)

cat_ordinal_pipe_v1 = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OrdinalEncoder(categories=[MSSubClass_scores, LotShape_scores, Utilities_scores, LotConfig_scores, LandSlope_scores, ExterQual_scores, ExterCond_scores, 
                                BsmtQual_scores, BsmtCond_scores, BsmtExposure_scores, BsmtFinType1_scores, BsmtFinType2_scores, HeatingQC_scores, KitchenQual_scores,
                                FireplaceQu_scores, GarageQual_scores, GarageCond_scores, PavedDrive_scores, PoolQC_scores]
                   )
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe_v1, num_columns_v1),
        ("cat_onehot_pipe", cat_onehot_pipe_v1, cat_onehot_calumns_v1),
        ("cat_ordinal_pipe", cat_ordinal_pipe_v1, cat_ordinal_calumns_v1)
    ]
)

# full_pipeline_v1 = make_pipeline(preprocessor, 
#                               DecisionTreeClassifier())

from sklearn.ensemble import RandomForestClassifier
full_pipeline_v1 = make_pipeline(preprocessor, 
                              RandomForestClassifier(max_depth= 12,min_samples_leaf= 3))
# from sklearn.svm import SVC
# full_pipeline_v1 = make_pipeline(preprocessor, 
#                               SVC())
# from sklearn.linear_model import LogisticRegression
# full_pipeline_v1 = make_pipeline(preprocessor, 
#                               LogisticRegression(max_iter= 100, solver= 'liblinear'))

# from sklearn.neighbors import KNeighborsClassifier
# full_pipeline_v1 = make_pipeline(preprocessor, 
#                               KNeighborsClassifier(leaf_size= 30, n_neighbors=  9, weights = 'uniform', algorithm= 'auto' ))
# from sklearn.gaussian_process import GaussianProcessClassifier
# full_pipeline_v1 = make_pipeline(preprocessor, 
#                               GaussianProcessClassifier())
# from sklearn.naive_bayes import GaussianNB
# full_pipeline_v1 = make_pipeline(preprocessor, 
#                                 GaussianNB())
# from sklearn.neural_network import MLPClassifier
# full_pipeline_v1 = make_pipeline(preprocessor, 
#                                 MLPClassifier())

# from sklearn.ensemble import AdaBoostClassifier
# full_pipeline_v1 = make_pipeline(preprocessor, 
#                                 AdaBoostClassifier())

best_param_1 = {'columntransformer__num_pipe__iterativeimputer__initial_strategy': 'mean',
 'randomforestclassifier__class_weight': 'balanced',
 'randomforestclassifier__criterion': 'gini',
 'randomforestclassifier__max_depth': 12,
 'randomforestclassifier__max_features': 'sqrt',
 'randomforestclassifier__min_samples_leaf': 3}

In [409]:
# Version 2

min_frequency= int(len(X)*0.002)
max_categories = int(len(X)*0.995)

numeric_pipe_v2 = make_pipeline(
    IterativeImputer(random_state=0, initial_strategy="mean"))
# create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
cat_onehot_pipe_v2 = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(min_frequency = min_frequency, max_categories=max_categories, handle_unknown='infrequent_if_exist', drop="first"),
)
cat_ordinal_pipe_v2 = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OrdinalEncoder(categories=[ExterQual_scores, ExterCond_scores, BsmtQual_scores, BsmtCond_scores, BsmtExposure_scores, BsmtFinType1_scores, BsmtFinType2_scores, HeatingQC_scores,
                               KitchenQual_scores, FireplaceQu_scores, GarageQual_scores,GarageCond_scores, PoolQC_scores]
                   )
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe_v2, num_columns_v2),
        ("cat_onehot_pipe", cat_onehot_pipe_v2, cat_onehot_calumns_v2),
        ("cat_ordinal_pipe", cat_ordinal_pipe_v2, cat_ordinal_calumns_v2)
    ]
)

# full_pipeline_v2 = make_pipeline(preprocessor, 
#                               DecisionTreeClassifier())

from sklearn.ensemble import RandomForestClassifier
full_pipeline_v2 = make_pipeline(preprocessor, 
                              RandomForestClassifier())

# from sklearn.svm import SVC
# full_pipeline_v2 = make_pipeline(preprocessor, 
#                               SVC())
# from sklearn.linear_model import LogisticRegression
# full_pipeline_v2 = make_pipeline(preprocessor, 
#                               LogisticRegression())
# from sklearn.neighbors import KNeighborsClassifier
# full_pipeline_v2 = make_pipeline(preprocessor, 
#                               KNeighborsClassifier())
# from sklearn.gaussian_process import GaussianProcessClassifier
# full_pipeline_v2 = make_pipeline(preprocessor, 
#                               GaussianProcessClassifier())
# from sklearn.naive_bayes import GaussianNB
# full_pipeline_v2 = make_pipeline(preprocessor, 
#                                 GaussianNB())
# from sklearn.neural_network import MLPClassifier
# full_pipeline_v2 = make_pipeline(preprocessor, 
#                                 MLPClassifier())
# from sklearn.ensemble import AdaBoostClassifier
# full_pipeline_v2 = make_pipeline(preprocessor, 
#                                 AdaBoostClassifier())

In [464]:
# Version 3

min_frequency= int(len(X)*0.002)
max_categories = int(len(X)*0.995)

numeric_pipe_v3 = make_pipeline(
    IterativeImputer(random_state=0, initial_strategy="mean"))
# create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
cat_onehot_pipe_v3 = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(min_frequency = min_frequency, max_categories=max_categories, handle_unknown='infrequent_if_exist', drop="first"),
)

cat_ordinal_pipe_v3 = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OrdinalEncoder(categories=[MSSubClass_scores, LotShape_scores, Utilities_scores, LotConfig_scores, LandSlope_scores, ExterQual_scores, ExterCond_scores, 
                                BsmtQual_scores, BsmtCond_scores, BsmtExposure_scores, BsmtFinType1_scores, BsmtFinType2_scores, HeatingQC_scores, KitchenQual_scores,
                                FireplaceQu_scores, GarageQual_scores, GarageCond_scores, PavedDrive_scores, PoolQC_scores]
                   )
)

all_scaler = make_pipeline(
    StandardScaler(),

)
preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe_v3, num_columns_v1),
        ("cat_onehot_pipe", cat_onehot_pipe_v3, cat_onehot_calumns_v1),
        ("cat_ordinal_pipe", cat_ordinal_pipe_v3, cat_ordinal_calumns_v1)
    ]
)


from sklearn.ensemble import RandomForestClassifier
full_pipeline_v3 = make_pipeline(preprocessor,
                                #  preprocessing.MinMaxScaler(), #best scaler till now
                                # preprocessing.FunctionTransformer(), # second best 
                                preprocessing.PolynomialFeatures(), # wowwwwwwwwwwwwwwwww
                              RandomForestClassifier(class_weight= 'balanced', max_depth= 12, min_samples_leaf= 3)
)


In [456]:
full_pipeline_v1.fit(df_filtered1, df_y1)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('iterativeimputer',
                                                                   IterativeImputer(random_state=0))]),
                                                  ['LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',...
                                                                                               'TA',
                                                                                               'Gd',
                                                                                               'Ex']]))]),
                                                  ['MSSubClass', 'LotShape',
                                                   'Utilities', 'LotConfig',
                                                   'LandSlope', 'ExterQual',
                                                   'ExterCond', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'HeatingQC',
                                                   'KitchenQual', 'FireplaceQu',
                                                   'GarageQual', 'GarageCond',
                                                   'PavedDrive', 'PoolQC'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=12, min_samples_leaf=3))])

In [403]:
full_pipeline_v2.fit(df_filtered1, df_y1)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('iterativeimputer',
                                                                   IterativeImputer(random_state=0))]),
                                                  ['LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',...
                                                                                               'Po',
                                                                                               'Fa',
                                                                                               'TA',
                                                                                               'Gd',
                                                                                               'Ex'],
                                                                                              ['NA',
                                                                                               'Po',
                                                                                               'Fa',
                                                                                               'TA',
                                                                                               'Gd',
                                                                                               'Ex'],
                                                                                              ['NA',
                                                                                               'Po',
                                                                                               'Fa',
                                                                                               'TA',
                                                                                               'Gd',
                                                                                               'Ex'],
                                                                                              ['NA',
                                                                                               'Po',
                                                                                               'Fa',
                                                                                               'TA',
                                                                                               'Gd',
                                                                                               'Ex']]))]),
                                                  ['ExterQual', 'ExterCond',
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'HeatingQC',
                                                   'KitchenQual', 'FireplaceQu',
                                                   'GarageQual', 'GarageCond',
                                                   'PoolQC'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [465]:
full_pipeline_v3.fit(df_filtered1, df_y1)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('iterativeimputer',
                                                                   IterativeImputer(random_state=0))]),
                                                  ['LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',...
                                                   'Utilities', 'LotConfig',
                                                   'LandSlope', 'ExterQual',
                                                   'ExterCond', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'HeatingQC',
                                                   'KitchenQual', 'FireplaceQu',
                                                   'GarageQual', 'GarageCond',
                                                   'PavedDrive', 'PoolQC'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=12,
                                        min_samples_leaf=3))])

In [457]:
forest_v124 = full_pipeline_v1.predict(test_df)

e:\Program Files\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 10, 16, 21, 23] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [405]:
tree_v202 = full_pipeline_v2.predict(test_df)

e:\Program Files\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 5, 15, 16, 22, 28] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [466]:
forest_v310 = full_pipeline_v3.predict(test_df)

e:\Program Files\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 10, 16, 21, 23] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [467]:
(
    test_df
        .assign(Expensive= forest_v310)
        .loc[:, ['Id', 'Expensive']]
        .to_csv('forest_v310.csv', index= False)
)

In [101]:
make_result(tree_v1).to_csv('tree_v1.csv', index=False)

In [102]:
make_result(tree_v1)

,Expensive,Id
147,0,148
676,0,677
1304,0,1305
1372,1,1373
1427,0,1428
...,...,...
1012,0,1013
1059,0,1060
890,0,891
628,0,629


In [91]:
X_test["Id"]

147      148
676      677
1304    1305
1372    1373
1427    1428
        ... 
1012    1013
1059    1060
890      891
628      629
352      353
Name: Id, Length: 292, dtype: int64

In [ ]:
import re
# select categorical and numerical column names
X_cat_columns = X.select_dtypes(exclude="number").columns
X_num_columns = X.select_dtypes(include="number").columns
X_cat_oneHot_columns = [X_cat_column for X_cat_column in X_cat_columns if not re.search("Cond$|Qu|^Bsmt", X_cat_column)]
X_cat_ordinal_columns = [X_cat_column for X_cat_column in X_cat_columns if re.search("Cond$|Qu|^Bsmt", X_cat_column)]

normal_scores = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
extra_scores = ['Ex', 'Gd', 'TA', 'Fa', 'Po' , 'NA']
BsmtExposure_scores = ['Gd', 'Av', 'Mn', 'No', 'NA']
BsmtFinType1_scores = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA']


min_frequency= int(len(X)*0.002)
max_categories = int(len(X)*0.995)

# create numerical pipeline, only with the IterativeImputer(random_state=0, initial_strategy="mean")
numeric_pipe = make_pipeline(
    IterativeImputer(random_state=0, initial_strategy="mean"))
 
 
 # create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
cat_onehot_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(min_frequency = min_frequency, max_categories=max_categories, handle_unknown='infrequent_if_exist', drop="first"),
)

cat_ordinal_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OrdinalEncoder(categories=[extra_scores, extra_scores, extra_scores, extra_scores, BsmtExposure_scores, BsmtFinType1_scores, extra_scores, extra_scores])
)
#Using ColumnTransformer a pipeline with 2 branches (the preprocessor):

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num_columns),
        ("cat_onehot_pipe", cat_onehot_pipe, X_cat_oneHot_columns),
        ("cat_ordinal_pipe", cat_ordinal_pipe, X_cat_ordinal_columns)
    ]
)

full_pipeline = make_pipeline(preprocessor, 
                              DecisionTreeClassifier())
full_pipeline.fit(X_train, y_train)


param_grid = {    
    "columntransformer__num_pipe__iterativeimputer__initial_strategy":["mean", "median"],
    "decisiontreeclassifier__max_depth": range(2, 14),
    "decisiontreeclassifier__min_samples_leaf": range(3, 12)
}

search = GridSearchCV(full_pipeline0,
                      param_grid0,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)